In [1]:
import sage
import numpy as np
var('k')

k

In [126]:
# given an even value of t, and a variable z which is the index of the last period whose coefficient we are considering
# returns a list of the coefficients of the first ceil(3m/2) periods from the relation given by 3t 
def ES3(t, z):
    poly_coeffs = []
    for l in range(1, z+1):
        coeff = 0
        if l%2==1:
            # since r_l correspond the r_{k-2-l} term by 1_t, the term in the summation (r_{k-2-t+l}) corresponds to
            # r_{t-l}. Thus, to find the coefficient of r_l, the index of the first summation must be t-l
            if m <= t:
                coeff = binomial(t,t-l)
            
            # since r_l corresponds to r_{k-2-l}, the two indices we care about are l and k-2-l, and the r_k-2-l
            # only appears if l>= t
            if l>=t:
                coeff += binomial(k-2-t, k-2-l)
            coeff += binomial(k-2-t,l)
            
            poly_coeffs.append(coeff)
    return poly_coeffs

In [125]:
# given integers m and a, returns the ceil(3m/2) square matrix A directly from the calculations of 3t 
def matrix_A_actual(m,a):
    coeffs = []
    matrix_size = (6*m + 2*floor(a/2) -2)/2
    for t in range(2*(matrix_size - ceil(3*m/2)) + 1,2*(matrix_size) + 1):
        if t%2 == 0:
            row = ES3(t,2*(matrix_size))
            for i in range(len(row)):
                coeffs.append(row[i])
    M = matrix(SR, ceil(3*m/2), matrix_size, coeffs)
    M = M.subs(k = 12*m+2*a)
    return M.matrix_from_rows_and_columns(range(ceil(3*m/2)), range(ceil(3*m/2)))

In [127]:
# given integers m and a, returns the ceiling(3m/2) square matrix A using the formula provided in the paper 
def matrix_A_formula(m, a):
    M = matrix(SR, ceil(3*m/2))
    for i in range(1, ceil(3*m/2)+1):
        for j in range(1, ceil(3*m/2)+1):
            M[i-1, j-1] =(binomial(2*a - 2*i + 6*m + 2*ceil(3/2*m) - 2*floor(1/2*a), 2*j - 1) 
                + binomial(2*i + 6*m - 2*ceil(3/2*m) + 2*floor(1/2*a) - 2, 2*j - 1))
            if m%2 == 1 and (a==1 or a==0)and i == 1 and j == ceil(3*m/2):
                M[i-1, j-1] = 9*m + binomial(9*m - 1 + 2*a, 3*m) - 1 + 2*a
    return M

In [131]:
# given integers m and a, returns the elimination matrix P that is used to get M into the form of a lower 
# anti-triangular matrix using the formula provided in the paper 
def matrix_P(m, a):
    P = matrix(SR, ceil(3*m/2))
    for i in range(1, ceil(3*m/2)+1):
        for j in range(1, ceil(3*m/2)+1):
            if i<=j:
                if a%2 == 0:
                    P[i-1,j-1] = (-1)^(-i + j)*(binomial(-2*i + 2*ceil(3/2*m) - 1, -i + j) - binomial(-2*i + 2*ceil(3/2*m) - 1, -i + j - 2))
                else:
                    P[i-1,j-1] = (-1)^(j-i)*2*(ceil(3*m/2) - j + 1)^2/((2*ceil(3*m/2) - i - j + 2)*(ceil(3*m/2) - i + 1))
                    P[i-1, j-1] *= binomial(2*ceil(3*m/2) - 2*i + 1,j - i)
            else:
                P[i-1,j-1] = 0
    return P

In [133]:
# given integers m and a, returns the anti-diagonal of PA in a list by actually computing PA
def anti_diagonal_actual(m,a):
    P = matrix_P_formula(m, a)
    A = matrix_A_formula(m, a)
    PA = P*A
    D = []
    for j in range(1, ceil(3*m/2)+1):
        y = ceil(3*m/2)+1-j
        D.append(PA[y-1, j-1])
    return D

In [135]:
# given integers m and a, returns the anti-diagonal of PA in a list using the formula provided in the paper 
def anti_diagonal_formula(m, a):
    D = []
    for j in range(1, ceil(3*m/2)+1):
        if a%2 ==0:
            D.append(2^(2*(j-1))*(12*m -2*j+2*a))
            if m%2==1 and j == (3*m+1)/2 and a == 0:
                D[j-1] = 2^(3*m-1)*(9*m -1) + (9*m-1)
        else:
            D.append(2^(2*(j-1))*(12*m -2*j+2*a)*(2*j-1)/j)
            if m%2==1 and j == (3*m+1)/2 and a == 1:
                D[j-1] = 2^(3*m)*(9*m +1)*(3*m)/(3*m+1) + (9*m+1)
    diagonal_term = []
    return D

In [138]:
m = 5
a = 1
print("m = ", m, "and", "a = ", a)
A_actual = matrix_A_actual(m,a)
A_formula = matrix_A_formula(m, a)
print("The matrix A given by the process described in the paper")
show(A_actual)
print()
print("The matrix A given by the formula provided in the paper")
show(A_formula)
print()
print("Returns if the actual matrix A and the formula match")
print(A_actual == A_formula)

m =  5 and a =  1
The matrix A given by the process described in the paper


[          60        15544      1372756     53528112   1101718332  13340783560 101766230804 511738760590]
[          60        13804      1090376     38332008    708941948   7669343500  51915526992 229911617072]
[          60        12296       859236     27010152    445940430   4280593200  25518739848  98672428432]
[          60        11020       673512     18721080    273606840   2311969400  12033300400  40225360560]
[          60         9976       528276     12790800    163509060   1204027720   5415447716  15471457104]
[          60         9164       419496      8693784     95450784    603301440   2313285744   5569210064]
[          60         8584       344036      6037416     55575806    293823920    938384360   1863693680]
[          60         8236       299656      4549896     34955700    150498660    384815760    603164880]


The matrix A given by the formula provided in the paper


[          60        15544      1372756     53528112   1101718332  13340783560 101766230804 511738760590]
[          60        13804      1090376     38332008    708941948   7669343500  51915526992 229911617072]
[          60        12296       859236     27010152    445940430   4280593200  25518739848  98672428432]
[          60        11020       673512     18721080    273606840   2311969400  12033300400  40225360560]
[          60         9976       528276     12790800    163509060   1204027720   5415447716  15471457104]
[          60         9164       419496      8693784     95450784    603301440   2313285744   5569210064]
[          60         8584       344036      6037416     55575806    293823920    938384360   1863693680]
[          60         8236       299656      4549896     34955700    150498660    384815760    603164880]


Returns if the actual matrix A and the formula match
True


In [139]:
m = 5
a = 1
print("m = ", m, "and", "a = ", a)
P = matrix_P_formula(m, a)
print("The matrix P given by the formula provided in the paper")
show(P)
A = matrix_A_formula(m, a)
print()
print("The matrix PA")
show(P*A)
print()

m =  5 and a =  1
The matrix P given by the formula provided in the paper


[      1   -49/4   135/2  -875/4     455 -2457/4  1001/2  -715/4]
[      0       1   -72/7   325/7  -832/7  1287/7 -1144/7   429/7]
[      0       0       1   -25/3    88/3     -55      55     -22]
[      0       0       0       1   -32/5    81/5   -96/5    42/5]
[      0       0       0       0       1    -9/2       7    -7/2]
[      0       0       0       0       0       1    -8/3     5/3]
[      0       0       0       0       0       0       1      -1]
[      0       0       0       0       0       0       0       1]


The matrix PA


[           0            0            0            0            0            0            0      1413166]
[           0            0            0            0            0            0    2555904/7  221859840/7]
[           0            0            0            0            0     281600/3   26540032/3    217470336]
[           0            0            0            0     119808/5      2444800  325683712/5 3564522336/5]
[           0            0            0         6048       666224     19193370    227497228   1344790496]
[           0            0       4480/3       177168   16524404/3  211806260/3 1356861152/3 4813905152/3]
[           0          348        44380      1487520     20620106    143325260    553568600   1260528800]
[          60         8236       299656      4549896     34955700    150498660    384815760    603164880]

In [140]:
m = 5
a = 1
print("m = ", m, "and", "a = ", a)
diagonal_actual = anti_diagonal_actual(m,a)
diagonal_formula = anti_diagonal_formula(m,a)
print("The list of the anti-diagonals taken from the matrix PA")
show(diagonal_actual)
print()
print("The list of the anti-diagonals given the formula provided in the paper")
show(diagonal_formula)
print()
print("Returns if the actual anti_diagonal and the formula match")
print(diagonal_actual == diagonal_formula)

m =  5 and a =  1
The list of the anti-diagonals taken from the matrix PA


[60, 348, 4480/3, 6048, 119808/5, 281600/3, 2555904/7, 1413166]


The list of the anti-diagonals given the formula provided in the paper


[60, 348, 4480/3, 6048, 119808/5, 281600/3, 2555904/7, 1413166]


Returns if the actual anti_diagonal and the formula match
True
